In [9]:
import pandas as pd
from darts import TimeSeries
import numpy as np



import warnings
warnings.filterwarnings("ignore")

def fix_datetime(X, name):
    """
    Function to fix and standardize datetime in the given DataFrame.
    
    Parameters:
    - X: DataFrame to be modified.
    - name: String representing the name of the DataFrame, used for logging.
    
    Returns:
    - Modified DataFrame with standardized datetime.
    """

    # Convert 'date_forecast' to datetime format and replace original column with 'ds'
    X['ds'] = pd.to_datetime(X['date_forecast'])
    X.drop(columns=['date_forecast'], inplace=True, errors='ignore')

    # Sort DataFrame by the new datetime column ('ds') and set it as the index
    X.sort_values(by='ds', inplace=True)
    X.set_index('ds', inplace=True)

    # Log the shape of the DataFrame before dropping rows with in-between minutes
    print(f"Shape of {name} before dropping in-between hour rows: ", X.shape)

    # Identify and log gaps in the date sequence
    print(f"HEIHEI: {name} gaps in dates: ", X.index.to_series().diff().dt.total_seconds().gt(60*15).sum())
    print(f"HEIHEI: {name} first gap in dates: ", X[X.index.to_series().diff().dt.total_seconds().gt(60*15)==True].index[:1])

    # Calculate and log the size of each gap in the date sequence
    temp = X.index.to_series().diff().dt.total_seconds()
    if temp.shape[0] > 0:
        print(f"HEIHEI: {name} list of size (in days) of each gap: ", temp[temp.gt(60*15)].values / (60*60*24))
    
    # temporarily transform into darts time series to fill missing dates
    # get date_calc if date_calc is column in X
    temp_calc = None
    if "date_calc" in X.columns:
        temp_calc = X["date_calc"]
        X.drop(columns=['date_calc'], inplace=True)
    X = TimeSeries.from_dataframe(df=X, freq="15T", fill_missing_dates=True, fillna_value=None).pd_dataframe()
    if temp_calc is not None:
        X["date_calc"] = temp_calc

    print(f"HEIHEI: {name} gaps in dates after filling missing dates: ", X.index.to_series().diff().dt.total_seconds().gt(60*15).sum())


    # Drop rows where the minute part of the time is not 0
    X = X[X.index.minute == 0]

    # Log the shape of the DataFrame after dropping rows with in-between minutes
    print(f"Shape of {name} after dropping in-between hour rows: ", X.shape)

    return X



def convert_to_datetime(X_train_observed, X_train_estimated, X_test, y_train):
    X_train_observed = fix_datetime(X_train_observed, "X_train_observed")
    X_train_estimated = fix_datetime(X_train_estimated, "X_train_estimated")
    X_test = fix_datetime(X_test, "X_test")


    X_train_observed["estimated_diff_hours"] = 0
    X_train_estimated["estimated_diff_hours"] = (X_train_estimated.index - pd.to_datetime(X_train_estimated["date_calc"])).dt.total_seconds() / 3600.0
    X_test["estimated_diff_hours"] = (X_test.index - pd.to_datetime(X_test["date_calc"])).dt.total_seconds() / 3600.0

    X_train_estimated.drop(columns=['date_calc'], inplace=True)
    X_test.drop(columns=['date_calc'], inplace=True)

    y_train['ds'] = pd.to_datetime(y_train['time'])
    y_train.drop(columns=['time'], inplace=True)
    y_train.sort_values(by='ds', inplace=True)
    y_train.set_index('ds', inplace=True)

    return X_train_observed, X_train_estimated, X_test, y_train




# location_map = {
#     "A": 0,
#     "B": 1,
#     "C": 2
# }


def preprocess_data(X_train_observed, X_train_estimated, X_test, y_train, location):
    # convert to datetime
    X_train_observed, X_train_estimated, X_test, y_train = convert_to_datetime(X_train_observed, X_train_estimated, X_test, y_train)


    # # cast all columns to float64
    # X_train = X_train.astype('float64')
    # X_test = X_test.astype('float64')


    print(f"X_train_observed shape: {X_train_observed.shape}")
    print(f"X_train_estimated shape: {X_train_estimated.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_train shape: {y_train.shape}")



    y_train["y"] = y_train["pv_measurement"].astype('float64')
    y_train.drop(columns=['pv_measurement'], inplace=True)
    print("y_train columns: ", y_train.columns)

    # temporarily transform into darts time series to fill missing dates
    print("Shape of y_train before filling missing dates: ", y_train.shape)
    y_train = TimeSeries.from_dataframe(df=y_train, freq="H", fill_missing_dates=True, fillna_value=None).pd_dataframe()
    print("Shape of y_train after filling missing dates: ", y_train.shape)


    # number of gaps in X_train_observed + X_train_estimated before
    print(f"LOOK: Number of gaps in X_train_observed plus number of gaps in X_train_estimated before: ", X_train_observed.index.to_series().diff().dt.total_seconds().gt(3600).sum() + X_train_estimated.index.to_series().diff().dt.total_seconds().gt(3600).sum())
    X_train = pd.concat([X_train_observed, X_train_estimated])
    print(f"LOOK: Number of gaps in X_train_observed plus number of gaps in X_train_estimated after: ", X_train.index.to_series().diff().dt.total_seconds().gt(3600).sum())
    # print size of gaps in X_train
    temp = X_train.index.to_series().diff().dt.total_seconds()
    if temp.shape[0] > 0:
        print("LOOK: list of size (in days) of each gap: ", temp[temp.gt(3600)].values / (60*60*24))
    print("if the number is bigger after than before that means there is a gap in time between the observed and estimated training sets")

    # print info on dates in X_train, and if there are any missing dates
    print("X_train dates info: ", X_train.index.min(), X_train.index.max(), X_train.index.max() - X_train.index.min())
    print("X_test dates info: ", X_test.index.min(), X_test.index.max(), X_test.index.max() - X_test.index.min())
    print("y_train dates info: ", y_train.index.min(), y_train.index.max(), y_train.index.max() - y_train.index.min())

    # any gaps in dates?
    print("X_train gaps in dates: ", X_train.index.to_series().diff().dt.total_seconds().gt(3600).sum())
    print("X_test gaps in dates: ", X_test.index.to_series().diff().dt.total_seconds().gt(3600).sum())
    print("y_train gaps in dates: ", y_train.index.to_series().diff().dt.total_seconds().gt(3600).sum())

    # temporarily transform into darts time series to fill missing dates
    X_train = TimeSeries.from_dataframe(df=X_train, freq="H", fill_missing_dates=True, fillna_value=None).pd_dataframe()
    X_test = TimeSeries.from_dataframe(df=X_test, freq="H", fill_missing_dates=True, fillna_value=None).pd_dataframe()
    print("X_train gaps in dates after filling missing dates: ", X_train.index.to_series().diff().dt.total_seconds().gt(3600).sum())
    print("X_test gaps in dates after filling missing dates: ", X_test.index.to_series().diff().dt.total_seconds().gt(3600).sum())

    

    # clip all y values to 0 if negative
    y_train["y"] = y_train["y"].clip(lower=0)
    
    # print Number of missing values in X train
    print("Number of missing values in X_train: ", X_train.isnull().sum().sum())
    print("Number of missing values in X_test: ", X_test.isnull().sum().sum())
    # y_train missing values
    print("Number of missing values in y_train: ", y_train.isnull().sum().sum())
    X_train = pd.merge(X_train, y_train, how="outer", left_index=True, right_index=True)
    print("Number of missing values in X_train after merging with y_train: ", X_train.drop(columns=['y']).isnull().sum().sum())



    X_train["location"] = location
    X_test["location"] = location
    
    return X_train, X_test
    


# Define locations
locations = ['A', 'B', 'C']

X_trains = []
X_tests = []
y_trains = []
# Loop through locations
for loc in locations:
    print("\n\n")
    print(f"Processing location {loc}...")
    # Read target training data
    y_train = pd.read_parquet(f'{loc}/train_targets.parquet')
    
    # Read estimated training data and add location feature
    X_train_estimated = pd.read_parquet(f'{loc}/X_train_estimated.parquet')
    
    # Read observed training data and add location feature
    X_train_observed= pd.read_parquet(f'{loc}/X_train_observed.parquet')

    # Read estimated test data and add location feature
    X_test_estimated = pd.read_parquet(f'{loc}/X_test_estimated.parquet')
    
    # Concatenate observed and estimated datasets for each location
    #X_train = pd.concat([X_train_estimated, X_train_observed])
    



    # Preprocess data
    X_train, X_test = preprocess_data(X_train_observed, X_train_estimated, X_test_estimated, y_train, loc)

    print(f"Final shape of X_train for location {loc}: ", X_train.shape)
    print(f"Final shape of X_test for location {loc}: ", X_test.shape)

    # print(y_train.head(), y_train.shape)
    # print(X_train.head(), X_train.shape)
    # print(X_train.head(), X_train.shape)
    # print(type(X_train['y']))

    # Save data to csv
    X_train.to_csv(f'{loc}/X_train.csv', index=True)
    X_test.to_csv(f'{loc}/X_test.csv', index=True)


    X_trains.append(X_train)
    X_tests.append(X_test)

# Concatenate all data and save to csv
X_train = pd.concat(X_trains)
X_test = pd.concat(X_tests)

print(f"Final shape of X_train: ", X_train.shape)
print(f"Final shape of X_test: ", X_test.shape)

X_train.to_csv('X_train_raw.csv', index=True)
X_test.to_csv('X_test_raw.csv', index=True)


# save where nan y values are dropped
X_train.dropna(subset=['y'], inplace=True)
X_train.to_csv('X_train.csv', index=True)




Processing location A...
Shape of X_train_observed before dropping in-between hour rows:  (118669, 45)
HEIHEI: X_train_observed gaps in dates:  0
HEIHEI: X_train_observed first gap in dates:  DatetimeIndex([], dtype='datetime64[ns]', name='ds', freq=None)
HEIHEI: X_train_observed list of size (in days) of each gap:  []
HEIHEI: X_train_observed gaps in dates after filling missing dates:  0
Shape of X_train_observed after dropping in-between hour rows:  (29668, 45)
Shape of X_train_estimated before dropping in-between hour rows:  (17576, 46)
HEIHEI: X_train_estimated gaps in dates:  1
HEIHEI: X_train_estimated first gap in dates:  DatetimeIndex(['2023-01-27'], dtype='datetime64[ns]', name='ds', freq=None)
HEIHEI: X_train_estimated list of size (in days) of each gap:  [1.01041667]
HEIHEI: X_train_estimated gaps in dates after filling missing dates:  0
Shape of X_train_estimated after dropping in-between hour rows:  (4418, 46)
Shape of X_test before dropping in-between hour rows:  (2880

# Starting

In [10]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset('X_train.csv')
test_data = TabularDataset('X_test_raw.csv')
label = 'y'

# mean absolute error
metric = 'mean_absolute_error'
time_limit = 60*60*4
predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')

Loaded data from: X_train.csv | Columns = 49 / 49 | Rows = 93024 -> 93024
Loaded data from: X_test_raw.csv | Columns = 48 / 48 | Rows = 4608 -> 4608
No path specified. Models will be saved in: "AutogluonModels/ag-20231005_083216/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 14400s
AutoGluon will save models to "AutogluonModels/ag-20231005_083216/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.179-5 (2023-08-08)
Disk Space Avail:   102.58 GB / 105.09 GB (97.6%)
Train Data Rows:    93024
Train Data Columns: 48
Label Column: y
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, -0.0, 287.01965, 766.40778)
	If

# Submit

In [11]:
# pull in test.csv and create submission.csv
submission_ids_df = pd.read_csv('test.csv', index_col=0)
# submission_ids_df has Id, Time, Location
# X_test has location_A, location_B, location_C
# we have to make sure that submission_ids_df has same dates as X_test, then predict y for each row in submission_ids_df, then save as submission.csv (using correct Id from submission_ids_df)

# convert index to datetime
submission_ids_df["id"] = submission_ids_df.index
submission_ids_df.set_index('time', inplace=True)
submission_ids_df.index = pd.to_datetime(submission_ids_df.index).astype('datetime64[ns]')
submission_ids_df

# split submission_ids_df into each location
print(submission_ids_df.groupby('location').count())


          prediction   id
location                 
A                720  720
B                720  720
C                720  720


In [12]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Assuming X_test and model are already defined
# Assuming 'time' is also a datetime64[ns] column in X_test


test_data = TabularDataset('X_test_raw.csv')
test_ids = TabularDataset('test.csv')
# merge test_data with test_ids
test_data = pd.merge(test_data, test_ids, how="inner", right_on=["time", "location"], left_on=["ds", "location"])

test_data

Loaded data from: X_test_raw.csv | Columns = 48 / 48 | Rows = 4608 -> 4608
Loaded data from: test.csv | Columns = 4 / 4 | Rows = 2160 -> 2160


,ds,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,estimated_diff_hours,location,id,time,prediction
0,2023-05-01 00:00:00,4.4,1.286,912.700012,0.000000,0.000000,1041.199951,0.0,271.700012,0.000000,...,30210.699219,4.0,2.2,3.4,-0.0,16.998889,A,0,2023-05-01 00:00:00,0
1,2023-05-01 01:00:00,4.3,1.287,NaN,0.000000,0.000000,1095.400024,0.0,271.600006,0.000000,...,29507.500000,3.9,2.0,3.3,-0.0,17.998889,A,1,2023-05-01 01:00:00,0
2,2023-05-01 02:00:00,4.2,1.284,1482.099976,0.000000,0.000000,1041.300049,0.0,271.200012,0.000000,...,29463.099609,3.7,1.8,3.2,-0.0,18.998889,A,2,2023-05-01 02:00:00,0
3,2023-05-01 03:00:00,4.1,1.282,2306.699951,0.000000,0.000000,1465.599976,0.0,270.799988,0.000000,...,33727.101562,3.6,1.6,3.2,-0.0,19.998889,A,3,2023-05-01 03:00:00,0
4,2023-05-01 04:00:00,3.9,1.282,2323.199951,59774.500000,43.000000,703.599976,0.0,270.299988,31.400000,...,35927.601562,3.4,1.3,3.1,-0.0,20.998889,A,4,2023-05-01 04:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2023-07-03 19:00:00,8.3,1.196,3532.399902,615338.812500,117.199997,2239.600098,0.0,281.600006,41.299999,...,41536.398438,2.2,1.9,-1.2,0.0,35.991389,C,2155,2023-07-03 19:00:00,0
2156,2023-07-03 20:00:00,8.5,1.199,3429.000000,269582.406250,40.000000,1513.699951,0.0,281.899994,19.700001,...,40136.500000,2.1,1.9,-0.9,0.0,36.991389,C,2156,2023-07-03 20:00:00,0
2157,2023-07-03 21:00:00,8.8,1.202,2495.000000,71999.601562,4.900000,1342.500000,0.0,282.200012,5.000000,...,43266.101562,2.4,2.2,-1.0,0.0,37.991389,C,2157,2023-07-03 21:00:00,0
2158,2023-07-03 22:00:00,9.0,1.206,1997.400024,1378.300049,0.000000,1878.900024,0.0,282.600006,0.000000,...,39017.898438,2.0,1.8,-0.8,0.0,38.991389,C,2158,2023-07-03 22:00:00,0


In [13]:
# predict
y_pred = predictor.predict(test_data)

In [14]:
y_pred

0        0.049316
1        0.054726
2        0.315490
3       13.609836
4       54.420658
          ...    
2155    24.931078
2156    21.207546
2157    17.164587
2158     9.467060
2159     6.092224
Name: y, Length: 2160, dtype: float32

In [15]:
# save y_pred series to csv as "id", "prediction"
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.rename(columns={"y": "prediction"}, inplace=True)
y_pred_df["id"] = y_pred_df.index
y_pred_df

,prediction,id
0,0.049316,0
1,0.054726,1
2,0.315490,2
3,13.609836,3
4,54.420658,4
...,...,...
2155,24.931078,2155
2156,21.207546,2156
2157,17.164587,2157
2158,9.467060,2158


In [16]:



# Save the submission DataFrame to submissions folder, create new name based on last submission, format is submission_<last_submission_number + 1>.csv
# Get the last submission number
last_submission_number = int(max([int(filename.split('_')[1].split('.')[0]) for filename in os.listdir('submissions') if "submission" in filename]))
print("Last submission number:", last_submission_number)

# Create the new filename
new_filename = f'submission_{last_submission_number + 1}.csv'

# Save the submission
y_pred_df.to_csv(os.path.join('submissions', new_filename), index=False)

Last submission number: 62
